In [1]:
import pandas as pd
import numpy as np 
import os 

In [2]:
path_test_directory = './Data/Test/'
path_tls_csv = os.path.join(path_test_directory, 'tls_test_01.pcapng.csv') 
path_tls_reassemble_csv = os.path.join(path_test_directory, 'reassemble_tls_test_01.pcapng.csv')

In [4]:
df_tls = pd.read_csv(path_tls_csv) 
df_reassemble_tls = pd.read_csv(path_tls_reassemble_csv) 
print(df_tls.shape) 
print(df_reassemble_tls.shape) 

(1983, 263)
(1983, 2)


In [5]:
df_merge_tls = pd.merge(df_tls, df_reassemble_tls, on=["index"], how="outer") 
df_merge_tls.shape 

(1983, 264)

In [7]:
df_merge_tls.to_csv(os.path.join(path_test_directory, "merge_tls_test.csv")) 

In [6]:
df_tls_csv.describe()

,Unnamed: 0,stream,completeness,completeness_rst,completeness_fin,completeness_data,completeness_ack,completeness_syn_ack,completeness_syn,completeness_str,...,handshake_server_curve_type,handshake_server_point_len,handshake_sig_len,handshake_client_point_len,analysis_spurious_retransmission,reassembled_in,options_sack_dsack,resumed,alert_message,analysis_fast_retransmission
count,1983.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,9.000000,9.0,9.000000,0.0,0.0,0.0,0.0,0.0,0.0
mean,991.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,31.666667,100.0,31.666667,NaN,NaN,NaN,NaN,NaN,NaN
std,572.587111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,11.067972,0.0,11.067972,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,20.000000,100.0,20.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,495.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,20.000000,100.0,20.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,991.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,41.000000,100.0,41.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,1486.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,41.000000,100.0,41.000000,NaN,NaN,NaN,NaN,NaN,NaN
max,1982.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,41.000000,100.0,41.000000,NaN,NaN,NaN,NaN,NaN,NaN
